**Ingesta y normalización de los datos**

*Importamos las librerias*

In [2]:
import pandas as pd
import numpy as np


*Ingestamos de archivos en formato csv y json*

In [3]:
df_amazon = pd.read_csv("./Datasets/amazon_prime_titles.csv")
df_disney=pd.read_csv("./Datasets/disney_plus_titles.csv")
df_hulu= pd.read_csv("./Datasets/hulu_titles.csv")
df_netflix=pd.read_json("./Datasets/netflix_titles.json")

In [4]:
#Agragamos a cada dataframe una columna con el nombre de la plataforma a la que pertenece
amazon=df_amazon.assign(Plataforma= "Amazon")
disney=df_disney.assign(Plataforma= "Disney")
hulu=df_hulu.assign(Plataforma= "Hulu")
netflix=df_netflix.assign(Plataforma= "Netflix")


*Creamos las funciones que utilizaremos a lo largo del proceso*

In [103]:
#Función para crear un dataframe que contenga los datasets ingestados anteriormente.
def concat(lista_dataframe):
    if type(lista_dataframe)== list: 
        df_concatenado=pd.concat(lista_dataframe)
        return df_concatenado
        
#Función para detirminar la cantidad de nulos de un dataframe o una columna de este.
def cant_null(dataframe):
    return dataframe.isna().sum()

In [6]:
#Instancio el dataframe al cual se le realizará el proceso de ETL
df=concat([amazon,disney,hulu,netflix])

*Estudiamos el tipo de dato de cada columna*

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 22998 entries, 0 to 8806
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   show_id       22998 non-null  object
 1   type          22998 non-null  object
 2   title         22998 non-null  object
 3   director      14739 non-null  object
 4   cast          17677 non-null  object
 5   country       11499 non-null  object
 6   date_added    13444 non-null  object
 7   release_year  22998 non-null  int64 
 8   rating        22134 non-null  object
 9   duration      22516 non-null  object
 10  listed_in     22998 non-null  object
 11  description   22994 non-null  object
 12  Plataforma    22998 non-null  object
dtypes: int64(1), object(12)
memory usage: 2.5+ MB


*Creamos una columna que indica el indice del dataframe concatenado(df)*

In [8]:
df = df.rename_axis('index').reset_index()
df

,index,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description,Plataforma
0,0,s1,Movie,The Grand Seduction,Don McKellar,"Brendan Gleeson, Taylor Kitsch, Gordon Pinsent",Canada,"March 30, 2021",2014,NaN,113 min,"Comedy, Drama",A small fishing village must procure a local d...,Amazon
1,1,s2,Movie,Take Care Good Night,Girish Joshi,"Mahesh Manjrekar, Abhay Mahajan, Sachin Khedekar",India,"March 30, 2021",2018,13+,110 min,"Drama, International",A Metro Family decides to fight a Cyber Crimin...,Amazon
2,2,s3,Movie,Secrets of Deception,Josh Webber,"Tom Sizemore, Lorenzo Lamas, Robert LaSardo, R...",United States,"March 30, 2021",2017,NaN,74 min,"Action, Drama, Suspense",After a man discovers his wife is cheating on ...,Amazon
3,3,s4,Movie,Pink: Staying True,Sonia Anderson,"Interviews with: Pink, Adele, Beyoncé, Britney...",United States,"March 30, 2021",2014,NaN,69 min,Documentary,"Pink breaks the mold once again, bringing her ...",Amazon
4,4,s5,Movie,Monster Maker,Giles Foster,"Harry Dean Stanton, Kieran O'Brien, George Cos...",United Kingdom,"March 30, 2021",1989,NaN,45 min,"Drama, Fantasy",Teenage Matt Banting wants to work with a famo...,Amazon
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22993,8802,s8803,Movie,Zodiac,David Fincher,"Mark Ruffalo, Jake Gyllenhaal, Robert Downey J...",United States,"November 20, 2019",2007,R,158 min,"Cult Movies, Dramas, Thrillers","A political cartoonist, a crime reporter and a...",Netflix
22994,8803,s8804,TV Show,Zombie Dumb,None,None,None,"July 1, 2019",2018,TV-Y7,2 Seasons,"Kids' TV, Korean TV Shows, TV Comedies","While living alone in a spooky town, a young g...",Netflix
22995,8804,s8805,Movie,Zombieland,Ruben Fleischer,"Jesse Eisenberg, Woody Harrelson, Emma Stone, ...",United States,"November 1, 2019",2009,R,88 min,"Comedies, Horror Movies",Looking to survive in a world taken over by zo...,Netflix
22996,8805,s8806,Movie,Zoom,Peter Hewitt,"Tim Allen, Courteney Cox, Chevy Chase, Kate Ma...",United States,"January 11, 2020",2006,PG,88 min,"Children & Family Movies, Comedies","Dragged from civilian life, a former superhero...",Netflix


*Separamos la columna "duration" , para crear las columnas 'time' y 'min_or_season'*

In [9]:
duration = df["duration"].str.split(expand=True)
duration.columns = ['time', 'min_or_season']
df= pd.concat([df, duration], axis=1)
df

,index,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description,Plataforma,time,min_or_season
0,0,s1,Movie,The Grand Seduction,Don McKellar,"Brendan Gleeson, Taylor Kitsch, Gordon Pinsent",Canada,"March 30, 2021",2014,NaN,113 min,"Comedy, Drama",A small fishing village must procure a local d...,Amazon,113,min
1,1,s2,Movie,Take Care Good Night,Girish Joshi,"Mahesh Manjrekar, Abhay Mahajan, Sachin Khedekar",India,"March 30, 2021",2018,13+,110 min,"Drama, International",A Metro Family decides to fight a Cyber Crimin...,Amazon,110,min
2,2,s3,Movie,Secrets of Deception,Josh Webber,"Tom Sizemore, Lorenzo Lamas, Robert LaSardo, R...",United States,"March 30, 2021",2017,NaN,74 min,"Action, Drama, Suspense",After a man discovers his wife is cheating on ...,Amazon,74,min
3,3,s4,Movie,Pink: Staying True,Sonia Anderson,"Interviews with: Pink, Adele, Beyoncé, Britney...",United States,"March 30, 2021",2014,NaN,69 min,Documentary,"Pink breaks the mold once again, bringing her ...",Amazon,69,min
4,4,s5,Movie,Monster Maker,Giles Foster,"Harry Dean Stanton, Kieran O'Brien, George Cos...",United Kingdom,"March 30, 2021",1989,NaN,45 min,"Drama, Fantasy",Teenage Matt Banting wants to work with a famo...,Amazon,45,min
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22993,8802,s8803,Movie,Zodiac,David Fincher,"Mark Ruffalo, Jake Gyllenhaal, Robert Downey J...",United States,"November 20, 2019",2007,R,158 min,"Cult Movies, Dramas, Thrillers","A political cartoonist, a crime reporter and a...",Netflix,158,min
22994,8803,s8804,TV Show,Zombie Dumb,None,None,None,"July 1, 2019",2018,TV-Y7,2 Seasons,"Kids' TV, Korean TV Shows, TV Comedies","While living alone in a spooky town, a young g...",Netflix,2,Seasons
22995,8804,s8805,Movie,Zombieland,Ruben Fleischer,"Jesse Eisenberg, Woody Harrelson, Emma Stone, ...",United States,"November 1, 2019",2009,R,88 min,"Comedies, Horror Movies",Looking to survive in a world taken over by zo...,Netflix,88,min
22996,8805,s8806,Movie,Zoom,Peter Hewitt,"Tim Allen, Courteney Cox, Chevy Chase, Kate Ma...",United States,"January 11, 2020",2006,PG,88 min,"Children & Family Movies, Comedies","Dragged from civilian life, a former superhero...",Netflix,88,min


*Convertimos la columna "time" a un tipo de dato entero*

In [10]:
#Reempelzamos los NaN por cero (0)
df['time']=df['time'].replace(np.nan, "0")

In [11]:
df['time']=df['time'].astype("int64")

In [12]:
cant_null(df['time'])

0

*Normalización de la columna min/season*

In [13]:
df['min_or_season'].value_counts()

min        15999
Season      4183
Seasons     2334
Name: min_or_season, dtype: int64

In [14]:
#Reemplazamos "Seasons" por "Season" en la colummna min_or_season
df["min_or_season"].replace("Seasons","Season", inplace=True)

In [15]:
df['min_or_season'].value_counts()


min       15999
Season     6517
Name: min_or_season, dtype: int64

*A continuación creamos la función para responder a la consulta:* Máxima duración según tipo de film (película/serie), por plataforma y por año: El request debe ser: get_max_duration(año, plataforma, [min o season])

In [119]:
def get_max_duration(anio, plataforma, min_or_season):
        df_rta=df[df['Plataforma'] == plataforma][df['release_year'] ==anio][df['min_or_season'] == min_or_season]
        rta=df_rta.groupby(['Plataforma', 'min_or_season', 'release_year'])['time'].idxmax()
        if min_or_season == "min":
            titulo_pelicula=df["title"].get(rta[0])
            return {f'La pelicula de mayor duración es : {titulo_pelicula}'}
        
        elif min_or_season == "Season":
            titulo_serie=df["title"].get(rta[0])
            return {f'La serie de mayor cantidad de temporadas es : {titulo_serie}'}
        else:
            return {'Los valores ingresados son incorrectos'}

In [120]:
get_max_duration(2018,'Hulu',"min")

<ipython-input-119-305b3edc46dd>:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df_rta=df[df['Plataforma'] == plataforma][df['release_year'] ==anio][df['min_or_season'] == min_or_season]


{'La pelicula de mayor duración es : The House That Jack Built'}

*A continuación creamos la función para responder a la consulta*: Cantidad de películas y series (separadas) por plataforma El request debe ser: get_count_plataform(plataforma)

In [134]:
def get_count_plataform(plataforma):
    df_type_por_plataforma=df[df["Plataforma"] == plataforma]
    df_type= df_type_por_plataforma.groupby(["type"])["type"].count()
    return df_type.to_dict()

In [135]:
get_count_plataform("Netflix")

{'Movie': 6131, 'TV Show': 2676}

*A continuación creamos la función para responder a la siguiente consulta* : Cantidad de veces que se repite un género y plataforma con mayor frecuencia del mismo. El request debe ser: get_listedin('genero')

In [20]:
def get_listedin(genero):
    df1=df[df["listed_in"].str.contains(f'{genero}')]
    Rta=df1.groupby(["Plataforma"])["Plataforma"].count()
    nombre_plataforma=Rta.idxmax()
    genero_repetido=Rta[0]
    return f'La plataforma en la que más se repite el género {genero} es {nombre_plataforma} apareciendo {genero_repetido}'

In [21]:
get_listedin("Comedy")

'La plataforma en la que más se repite el género Comedy es Amazon apareciendo 2099'

*A continuación creamos la función para responder a la siguiente consulta* : Actor que más se repite según plataforma y año. El request debe ser: get_actor(plataforma, año)

*1º Definimos un dataframe ,al que llamamos "prueba", que contiene las columnas necesarias para la consulta.*


In [24]:
prueba=pd.DataFrame({"año":df["release_year"],"plataforma":df["Plataforma"],"actor":df["cast"]})

In [ ]:
prueba

*2º Creamos un dataframe, al que llamamos "df_final", que con las siguientes columnas :*
*<br> **actor** que contiene los nombres de cada actor,para esto se utilizó la información de la columna : prueba["actor"],*
*<br>además de las columnas **año** y **plataforma**, que se tomáron del dataframe prueba*

In [ ]:
df_final=pd.DataFrame({"año":[],"plataforma":[],"actor":[]})
for i in range(0,len(prueba)):
 
    if type(prueba.loc[i]["actor"]) == str:
        actores=prueba.loc[i]["actor"].split(",")
        print(i)
        for j in range(0,len(actores)):
            nueva_fila = {"año":prueba.loc[i]["año"], "plataforma":prueba.loc[i]["plataforma"], "actor":actores[j]}
            df_final=df_final.append(nueva_fila, ignore_index=True)
df_final 



In [58]:
df_final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 114392 entries, 0 to 114391
Data columns (total 3 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   año         114392 non-null  float64
 1   plataforma  114392 non-null  object 
 2   actor       114392 non-null  object 
dtypes: float64(1), object(2)
memory usage: 2.6+ MB


In [59]:
#Cambiamos el tipo de dato de la columna año:
df_final['año']=df_final['año'].astype("int64")

In [ ]:
df_final.info()

*3º Finalmente definimos la función **get_actor(plataforma, anio)**:*

In [114]:
def get_actor(plataforma, anio):
    condicion=df_final[df_final["año"]==anio][df_final["plataforma"]==plataforma]
    nombre_actor=condicion.groupby(["actor"])["actor"].count().idxmax()
    veces_repetido=condicion.groupby(["actor"])["actor"].count().max()
    return f'El/la actor/actriz que más se repite en la plataforma {plataforma}, en el año {anio}, es: {nombre_actor} apareciendo {veces_repetido} veces'
 

In [115]:
get_actor("Netflix",2018)

<ipython-input-114-fe84f3745086>:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  condicion=df_final[df_final["año"]==anio][df_final["plataforma"]==plataforma]


'El/la actor/actriz que más se repite en la plataforma Netflix, en el año 2018, es:  Andrea Libman apareciendo 8 veces'

*Tranformamos los dataframe "df" y "df_final" en archivos csv*

In [62]:
df_final.to_csv('actores.csv')

In [101]:
df.to_csv('tabla_consultas.csv')